#  Blue Courses Analysis
##### This script reads the Blue courses CSV, filters to Fall 2025(the semester you choose), parses date fields, and fills in missing SOQ windows by setting EvaluationStart to 14 days before a course’s end date and EvaluationEnd to 1 day before the end date. It then groups the data by Part of Term to compute each session’s earliest start, latest end, and combined SOQ window, along with a list of unique course names offered in that session. Finally, it formats the dates (weekday + month + day), renames columns , and exports the result to Excel at NEWSOQCalender.xlsx. < you can name it based w.


In [ ]:
import pandas as pd
from datetime import timedelta

# reading from the O drive 
df = pd.read_csv(r'O:\data\Blue\BlueCourses.csv')

#HERE ... first change to desired semester ***
df = df[df['SemesterDesc'] == 'Fall 2025']

def format_date(date):
    if pd.notnull(date):
        return date.strftime('%A, %B %d')
    else:
        return ''

df['CourseStartDate'] = pd.to_datetime(df['CourseStartDate'])
df['CourseEndDate'] = pd.to_datetime(df['CourseEndDate'])
df['EvaluationStart'] = pd.to_datetime(df['EvaluationStart'], errors='coerce')
df['EvaluationEnd'] = pd.to_datetime(df['EvaluationEnd'], errors='coerce')

# Filling the NaN SOQ dates
def fill_soq_dates(row):
    if pd.isna(row['EvaluationStart']) or pd.isna(row['EvaluationEnd']):
        row['EvaluationStart'] = row['CourseEndDate'] - timedelta(days=14)
        row['EvaluationEnd'] = row['CourseEndDate'] - timedelta(days=1)
    return row

df = df.apply(fill_soq_dates, axis=1)

aggregated_df = df.groupby('Part_Of_Term').agg({
    'CourseStartDate': 'min',
    'CourseEndDate': 'max',
    'EvaluationStart': 'min',
    'EvaluationEnd': 'max',
    'CourseName': lambda x: ', '.join(x.unique())
}).reset_index()

aggregated_df['CourseStartDate'] = aggregated_df['CourseStartDate'].apply(format_date)
aggregated_df['CourseEndDate'] = aggregated_df['CourseEndDate'].apply(format_date)
aggregated_df['EvaluationStart'] = aggregated_df['EvaluationStart'].apply(format_date)
aggregated_df['EvaluationEnd'] = aggregated_df['EvaluationEnd'].apply(format_date)

# Renaming columns
aggregated_df.columns = ['Part Of Term', 'Course Start Date', 'Course End Date', 'SOQ Start', 'SOQ End', 'CourseName']

# Saving file
aggregated_df.to_excel(r'C:\Users\malsaegh\Desktop\WCC\SOQCalender\NEWSOQCalender.xlsx', index=False)

print("The file is saved as NEWSOQCalender.xlsx'")


The file is saved as NEWSOQCalender.xlsx'


### FINDING THE NUMBER OF POT PER SEMESTER 

In [19]:
import pandas as pd

df = pd.read_csv(r'O:\data\Blue\BlueCourses.csv')

part_of_term_counts = df.groupby('SemesterDesc')['Part_Of_Term'].nunique().reset_index()

part_of_term_counts.columns = ['SemesterDesc', 'Part_Of_Term_Count']

print(part_of_term_counts)


         SemesterDesc  Part_Of_Term_Count
0           Fall 2025                  33
1  Spring/Summer 2025                  32
2         Winter 2026                  30


# TESTING OTHER MISSING COURSES

In [4]:
import pandas as pd

# Load the Excel file
file_path = r'C:\Users\malsaegh\Downloads\ComparisonPointers.xlsx'

# Load the data from the sheets
sheet1 = pd.read_excel(file_path, sheet_name='Sheet1')
sheet2 = pd.read_excel(file_path, sheet_name='Sheet2')

# Extract the numeric part of the "Course Number" in Sheet1
sheet1['Numeric Course Number'] = sheet1['Course Number'].str.extract(r'W_(\d+)P')

# Get the unique course numbers from each sheet
sheet1_courses = sheet1['Numeric Course Number'].dropna().astype(int).unique()
sheet2_courses = sheet2['All Course Number'].dropna().astype(int).unique()

# Find courses in Sheet2 that are not in Sheet1
missing_courses = [course for course in sheet2_courses if course not in sheet1_courses]

# Create a DataFrame for the missing courses, including the full "W_...P" format
missing_courses_df = pd.DataFrame({
    'Missing Course Numbers': missing_courses,
    'Full Format': [f'W_{course}P' for course in missing_courses]
})

# Save the missing courses to a new Excel file
output_path = r'C:\Users\malsaegh\Downloads\MissingCoursesWithFormat.xlsx'
missing_courses_df.to_excel(output_path, index=False)

print(f"Missing courses have been written to {output_path}")


Missing courses have been written to C:\Users\malsaegh\Downloads\MissingCoursesWithFormat.xlsx


In [5]:
import pandas as pd
from docx import Document
from docx.shared import Inches, Pt
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.table import WD_TABLE_ALIGNMENT, WD_ALIGN_VERTICAL

# Load Excel file
excel_path = r'C:\Users\malsaegh\Desktop\WCC\SOQCalender\s2025_SpringSummerDatesSOQ.xlsx'
df = pd.read_excel(excel_path)

# Create a new Word document
doc = Document()

# Title and subtitle
doc.add_heading('Important Dates for Spring/Summer 2025', level=1)

para = doc.add_paragraph('Session Dates and\nStudent Opinion Questionnaire (SOQ) Dates ~ ALL SOQs are ONLINE')
para.alignment = WD_ALIGN_PARAGRAPH.LEFT

# Add spacing
doc.add_paragraph()

# Define table headers
columns = [
    'Part Of Term',
    'Course Start Date',
    'Course End Date',
    'SOQ Start',
    'SOQ End',
    'Session Length/Section number*'
]

# Add table
table = doc.add_table(rows=1, cols=len(columns))
table.style = 'Table Grid'
table.alignment = WD_TABLE_ALIGNMENT.LEFT
table.autofit = True

# Format header row
hdr_cells = table.rows[0].cells
for i, col_name in enumerate(columns):
    hdr_cells[i].text = col_name
    paragraph = hdr_cells[i].paragraphs[0]
    run = paragraph.runs[0]
    run.bold = True
    paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
    hdr_cells[i].vertical_alignment = WD_ALIGN_VERTICAL.CENTER

    # Apply gray background shading
    tc = hdr_cells[i]._tc
    tcPr = tc.get_or_add_tcPr()
    shd = OxmlElement('w:shd')
    shd.set(qn('w:fill'), 'D9D9D9')  # light gray
    shd.set(qn('w:val'), 'clear')
    shd.set(qn('w:color'), 'auto')
    tcPr.append(shd)

# Add data rows
for index, row in df.iterrows():
    cells = table.add_row().cells
    cells[0].text = str(row['Part Of Term'])
    cells[1].text = str(row['Course Start Date'])
    cells[2].text = str(row['Course End Date'])
    cells[3].text = str(row['SOQ Start'])
    cells[4].text = str(row['SOQ End'])
    cells[5].text = ''  # Leave last column blank as requested

    for cell in cells:
        cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        cell.paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.LEFT
        cell.paragraphs[0].runs[0].font.size = Pt(10.5)

# Save the styled Word document
output_path = r'C:\Users\malsaegh\Desktop\WCC\SOQCalender\s2025_fallSOQ.docx'
doc.save(output_path)

print(f"Styled Word document saved to: {output_path}")


Styled Word document saved to: C:\Users\malsaegh\Desktop\WCC\SOQCalender\s2025_fallSOQ.docx
